- **Purpose:** Follow the tutorial for multiGSEA
- **Date:** August 26, 2025

# Step 1
Load libraries and data

In [1]:
# Libraries
library(multiGSEA)
library(org.Hs.eg.db)
library(tidyverse)

loading from cache

Loading required package: AnnotationDbi

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, saveRDS, setdiff,
    table, tapply, union, unique, unsplit, which.max, which.min


Loading required package: Biobase

Welcome to Bioconductor

    Vignettes contain introductory material; view with
    'browseVignettes()'. To cite Bioconductor, see
    'citation("Biobase")', and for packages 'citation("pkgname")'.


Loading required package: IRanges

Loading requ

In [2]:
# Data
data(transcriptome) # transcriptomic data
data(proteome) # proteomic data
data(metabolome) # metabolomic data

# Step 2
Prepare data for enrichment analysis. Three columns must be present in the input the data -
1. Feature IDs
2. logFC
3. p value

In [3]:
# Transcriptomic data
head(transcriptome)

Symbol,logFC,pValue,adj.pValue
<chr>,<dbl>,<dbl>,<dbl>
STC2,0.7045003,3.710893e-14,5.630909e-10
ASNS,0.6279663,2.025533e-13,1.536772e-09
PCK2,0.6193553,7.774776e-13,3.932482e-09
FAM129A,0.8808264,1.069436e-12,4.056904e-09
NUPR1,0.5888412,1.546367e-12,4.692915e-09
ASS1,0.6241387,2.416958e-12,6.112487e-09


In [4]:
# Proteome data
head(proteome)

Symbol,logFC,pValue,adj.pValue
<chr>,<dbl>,<dbl>,<dbl>
IFRD1,2.171009,1.519770e-11,1.257610e-07
FAM129A,1.967014,7.793641e-11,3.224619e-07
FDFT1,-1.777090,2.493535e-10,6.877999e-07
ASNS,1.457231,4.708884e-10,9.741505e-07
CTH,1.573975,1.217737e-09,1.665428e-06
PCK2,1.381192,1.233280e-09,1.665428e-06


In [5]:
# Metabolome data
head(metabolome)

HMDB,logFC,pValue,adj.pValue
<chr>,<dbl>,<dbl>,<dbl>
HMDB00042,-0.01840632,0.0393850546,0.0715518519
HMDB03344,-0.01840632,0.0393850546,0.0715518519
HMDB00820,-0.08093436,0.0001297692,0.0004585449
HMDB00863,-0.08093436,0.0001297692,0.0004585449
HMDB06853,-0.08319954,0.0007402161,0.0021531643
HMDB13785,-0.08319954,0.0007402161,0.0021531643


In [6]:
# Initialize the data structure with multiple layers
omics_data <- initOmicsDataStructure(
    layer = c(
        "transcriptome",
        "proteome",
        "metabolome"
    )
)

In [7]:
# Add data to the omics data structure

# Transcriptomic data
omics_data$transcriptome <- rankFeatures(transcriptome$logFC, transcriptome$pValue)
names(omics_data$transcriptome) <- transcriptome$Symbol

# Proteomic data
omics_data$proteome <- rankFeatures(proteome$logFC, proteome$pValue)
names(omics_data$proteome) <- proteome$Symbol

# Metabolomic data
omics_data$metabolome <- rankFeatures(metabolome$logFC, metabolome$pValue)
names(omics_data$metabolome) <- metabolome$HMDB
names(omics_data$metabolome) <- gsub("HMDB", "HMD00", names(omics_data$metabolome))

In [8]:
# Look at the class of the omics data structure
class(omics_data)

[1] "list"

# Step 3
Pathway definition

In [9]:
# Listing layers and databases
layers <- names(omics_data)
databases <- c("kegg", "reactome")

In [10]:
# Get the pathway-related data from the database
pathways <- getMultiOmicsFeatures(
    dbs = databases,
    layer = layers,
    returnTranscriptome = "SYMBOL",
    returnProteome = "SYMBOL",
    returnMetabolome = "HMDB",
    useLocal = FALSE
) # graphite R package is used in this case 

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:many mapping between keys and columns

'select()' returned 1:ma

In [11]:
# Look at the pathways 
pathways_short <- lapply(names(pathways), function(name){head(pathways[[name]], 2)})
names(pathways_short) <- names(pathways)
pathways_short

$transcriptome
$transcriptome$`(KEGG) Glycolysis / Gluconeogenesis`
 [1] "AKR1A1"  "ADH1A"   "ADH1B"   "ADH1C"   "ADH4"    "ADH5"    "ADH6"   
 [8] "GALM"    "ADH7"    "LDHAL6A" "DLAT"    "DLD"     "ENO1"    "ENO2"   
[15] "ENO3"    "ALDH2"   "ALDH3A1" "ALDH1B1" "FBP1"    "ALDH3B1" "ALDH3B2"
[22] "ALDH9A1" "ALDH3A2" "ALDOA"   "ALDOB"   "ALDOC"   "G6PC1"   "GAPDH"  
[29] "GAPDHS"  "GCK"     "GPI"     "HK1"     "HK2"     "HK3"     "ENO4"   
[36] "LDHA"    "LDHB"    "LDHC"    "PGAM4"   "ALDH7A1" "PCK1"    "PCK2"   
[43] "PDHA1"   "PDHA2"   "PDHB"    "PFKL"    "PFKM"    "PFKP"    "PGAM1"  
[50] "PGAM2"   "PGK1"    "PGK2"    "PGM1"    "PKLR"    "PKM"     "PGM2"   
[57] "ACSS2"   "G6PC2"   "BPGM"    "TPI1"    "HKDC1"   "ADPGK"   "ACSS1"  
[64] "FBP2"    "LDHAL6B" "G6PC3"   "MINPP1" 

$transcriptome$`(KEGG) Citrate cycle (TCA cycle)`
 [1] "CS"     "DLAT"   "DLD"    "DLST"   "FH"     "IDH1"   "IDH2"   "IDH3A" 
 [9] "IDH3B"  "IDH3G"  "MDH1"   "MDH2"   "ACLY"   "ACO1"   "OGDH"   "ACO2"  
[17] "PC"     "PDHA1"  "PDHA2"  "PDHB"   "OGDHL"  "SDHA"   "SDHB"   "SDHC"  
[25] "SDHD"   "SUCLG2" "SUCLG1" "SUCLA2" "PCK1"   "PCK2"  


$proteome
$proteome$`(KEGG) Glycolysis / Gluconeogenesis`
 [1] "AKR1A1"  "ADH1A"   "ADH1B"   "ADH1C"   "ADH4"    "ADH5"    "ADH6"   
 [8] "GALM"    "ADH7"    "LDHAL6A" "DLAT"    "DLD"     "ENO1"    "ENO2"   
[15] "ENO3"    "ALDH2"   "ALDH3A1" "ALDH1B1" "FBP1"    "ALDH3B1" "ALDH3B2"
[22] "ALDH9A1" "ALDH3A2" "ALDOA"   "ALDOB"   "ALDOC"   "G6PC1"   "GAPDH"  
[29] "GAPDHS"  "GCK"     "GPI"     "HK1"     "HK2"     "HK3"     "ENO4"   
[36] "LDHA"    "LDHB"    "LDHC"    "PGAM4"   "ALDH7A1" "PCK1"    "PCK2"   
[43] "PDHA1"   "PDHA2"   "PDHB"    "PFKL"    "PFKM"    "PFKP"    "PGAM1"  
[50] "PGAM2"   "PGK1"    "PGK2"    "PGM1"    "PKLR"    "PKM"     "PGM2"   
[57] "ACSS2"   "G6PC2"   "BPGM"    "TPI1"    "HKDC1"   "ADPGK"   "ACSS1"  
[64] "FBP2"    "LDHAL6B" "G6PC3"   "MINPP1" 

$proteome$`(KEGG) Citrate cycle (TCA cycle)`
 [1] "CS"     "DLAT"   "DLD"    "DLST"   "FH"     "IDH1"   "IDH2"   "IDH3A" 
 [9] "IDH3B"  "IDH3G"  "MDH1"   "MDH2"   "ACLY"   "ACO1"   "OGDH"   "ACO2"  
[17] "PC"     "PDHA1"  "PDHA2"  "PDHB"   "OGDHL"  "SDHA"   "SDHB"   "SDHC"  
[25] "SDHD"   "SUCLG2" "SUCLG1" "SUCLA2" "PCK1"   "PCK2"  


$metabolome
$metabolome$`(KEGG) Glycolysis / Gluconeogenesis`
 [1] "HMDB0001586" "HMDB0001206" "HMDB0001294" "HMDB0001270" "HMDB0000122"
 [6] "HMDB0003498" "HMDB0003391" "HMDB0060180" "HMDB0003904" "HMDB0000108"
[11] "HMDB0000223" "HMDB0000243" "HMDB0000042" "HMDB0000190" "HMDB0000990"
[16] "HMDB0001473" "HMDB0003345" "HMDB0000263" "HMDB0001112" "HMDB0000124"

$metabolome$`(KEGG) Citrate cycle (TCA cycle)`
 [1] "HMDB0001206" "HMDB0003974" "HMDB0001022" "HMDB0006744" "HMDB0003904"
 [6] "HMDB0000094" "HMDB0000134" "HMDB0000223" "HMDB0000243" "HMDB0000254"
[11] "HMDB0000208" "HMDB0000263" "HMDB0000156" "HMDB0000072" "HMDB0000193"

# Step 4
Pathway enrichment analysis

In [12]:
# Enrichment score calculation 
enrichment_scores <- multiGSEA(pathways, omics_data)

Warning message in preparePathwaysAndStats(pathways, stats, minSize, maxSize, gseaParam, :
“There are ties in the preranked stats (79.1% of the list).
The order of those tied genes will be arbitrary, which may produce unexpected results.”


In [21]:
# Check the propotion of duplicated ranks
sum(duplicated(rankFeatures(metabolome$logFC, metabolome$pValue)))/nrow(metabolome)

[1] 0.7910264

*Conclusion: too many duplications. Results will thus be unreliable*

# Step 5
Aggregate pathway enrichment

In [13]:
df <- extractPvalues(
    enrichmentScores = enrichment_scores,
    pathwayNames = names(pathways[[1]])
)
df$combined_pval <- combinePvalues(df)
df$combined_padj <- p.adjust(df$combined_pval, method = "BH")
df <- cbind(data.frame(pathway = names(pathways[[1]])), df)
head(df)

,pathway,transcriptome_pval,transcriptome_padj,proteome_pval,proteome_padj,metabolome_pval,metabolome_padj,combined_pval,combined_padj
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,(KEGG) Glycolysis / Gluconeogenesis,0.3625000,0.6745815,5.858400e-04,0.009727304,NA,NA,0.005482761,0.04414727
2,(KEGG) Citrate cycle (TCA cycle),0.2564103,0.5922683,9.304550e-03,0.068241427,NA,NA,0.016719142,0.09833076
3,(KEGG) Pentose phosphate pathway,0.9322034,0.9811653,4.710889e-05,0.001387469,NA,NA,0.044006213,0.18822826
4,(KEGG) Pentose and glucuronate interconversions,0.9170306,0.9780260,1.061838e-03,0.015184902,NA,NA,0.116456885,0.33527463
5,(KEGG) Fructose and mannose metabolism,0.1810700,0.5143114,5.637329e-04,0.009423480,NA,NA,0.001603524,0.01658931
6,(KEGG) Galactose metabolism,0.6942149,0.8899517,2.549173e-02,0.138947404,NA,NA,0.153648390,0.39025060


*Too many duplications in the ranked metabolome data is interfering with correct inferences* 

In [14]:
sessionInfo()

R version 4.4.3 (2025-02-28)
Platform: x86_64-conda-linux-gnu
Running under: SUSE Linux Enterprise Server 15 SP6

Matrix products: default
BLAS/LAPACK: /nexus/posix0/MPI-psych/g/dept-binder/mpsmgp/ahsant/software/miniforge3/envs/jupyter/lib/libopenblasp-r0.3.28.so;  LAPACK version 3.12.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] lubridate_1.9.4      forcats_1.0.0        stringr_1.5.1       
 [4] dplyr_1.1.4          purrr_1.1.0          readr_2.1.5         
 [7] tidyr_1.3.1 